In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pymongo import MongoClient
from datetime import datetime

In [2]:
conn=sqlite3.connect('archive/FPA_FOD_20170508.sqlite')

In [3]:
df=pd.read_sql("""
SELECT * from fires""", con=conn)

In [9]:
df['CONT_TIME'].fillna("1200", inplace=True)
df['DISCOVERY_TIME'].fillna("1200", inplace=True)

In [10]:
epoch=pd.to_datetime(0, unit='s').to_julian_date()
def fix_time(date, time):
    
    if np.isnan(date):
         return None
    else:
        date=pd.to_datetime(date - epoch, unit='D').strftime("%Y-%m-%d")        
        return pd.to_datetime(date + ' ' + time)

#combine the dates and times into one column as a complete datetime
#df['DISCOVERY_DATE']=pd.to_datetime(df['DISCOVERY_DATE'].astype(str) + ' ' + df['DISCOVERY_TIME'].astype(str))
#df['CONT_DATE']=pd.to_datetime(df['CONT_DATE'].astype(str) + ' ' + df['CONT_TIME'].astype(str))

df['DISCOVERY_DATE']=df.apply(lambda x: fix_time(x['DISCOVERY_DATE'],x['DISCOVERY_TIME']), axis=1)

In [11]:
df['CONT_DATE']=df.apply(lambda x: fix_time(x['CONT_DATE'],x['CONT_TIME']), axis=1)

In [12]:
#df['CONT_DATE'].fillna("None", inplace=True)
#df['DISCOVERY_DATE'].fillna("None", inplace=True)

df['CONT_DATE'] = df['CONT_DATE'].astype(object).where(df['CONT_DATE'].notnull(), None)
df['DISCOVERY_DATE'] = df['DISCOVERY_DATE'].astype(object).where(df['DISCOVERY_DATE'].notnull(), None)

In [14]:
client=MongoClient('mongodb://famp.fishermenmedia.com:27017/')
db=client['fires']
firesCollection=db['fires']

In [15]:
df.reset_index(inplace=True)

In [16]:
df_dict=df.to_dict("records")

In [17]:
firesCollection.insert_many(df_dict)

In [18]:
firesCollection.count_documents({})

1880465